In [1]:
import simpy
import config
import random
import numpy as np
from numpy import random as np_random
import torch
import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical


In [2]:
class ManufacturingProcess:
    def __init__(self, env, name, capacity, process_time, conveyor_belt, stats):
        """
        Initializes a manufacturing process.
        
        Args:
            env (simpy.Environment): Simulation environment.
            name (str): Name of the process.
            capacity (int): Number of machines in the process.
            process_time (int or function): Fixed or stochastic processing time.
            conveyor_belt (int): Buffer size between stages.
            stats (dict): Dictionary to store simulation statistics.
        """
        self.env = env
        self.name = name
        self.machine = simpy.Resource(env, capacity)
        self.process_time = process_time
        self.conveyor_belt = conveyor_belt  # Buffer capacity
        self.stats = stats
        self.total_busy_time = 0  # Track total processing time

        # Store machine utilization per item
        self.stats['machine_utilization'][self.name] = 0

    def process(self, item):
        """
        Simulates the processing of an item.
        
        Args:
            item (int): The item ID being processed.
        """
        with self.machine.request() as request:
            request_start = self.env.now  # Time when machine is requested

            yield request  # Wait for machine availability

            start_time = self.env.now  # Processing start time
            process_duration = self.get_processing_time()
            yield self.env.timeout(process_duration)  # Process the item
            end_time = self.env.now  # Processing end time

            # Track total busy time
            self.total_busy_time += process_duration

            # Log machine activity
            print(f'{self.name} processed item {item} from {start_time} to {end_time} (Duration: {process_duration})')

            # Store machine utilization
            self.stats['machine_utilization'][self.name] += process_duration

    def get_processing_time(self):
        """
        Returns processing time, either fixed or stochastic.
        
        Returns:
            float: Processing time duration.
        """
        if isinstance(self.process_time, (int, float)):  # Fixed processing time
            return self.process_time
        elif callable(self.process_time):  # Stochastic processing time
            return self.process_time()


In [3]:
NUM_REPLICATIONS = 10  # 🔄 Number of replications
BASE_SEED = 42         # 🎯 Base seed for reproducibility (change for different overall outcomes)

In [ ]:
def run_simulation(replication_num, blow_molding_process_time, cleaning_process_time,
                    filling_process_time, capping_labeling_process_time, packaging_process_time):
    """
    Runs one replication of the simulation with a unique seed.
    """
    # 🎲 Set a unique seed for each replication
    random.seed(BASE_SEED + replication_num)

    env = simpy.Environment()

    # Statistics tracking for each replication
    stats = {
        'processed_items': 0,  # Total completed items
        'queue_lengths': [],
        'bottleneck_station': None,
        'machine_utilization': {}
    }

    # Define manufacturing stages
    blow_molding = ManufacturingProcess(env, "Blow Molding", config.MACHINE_CAPACITIES[0],
                                        blow_molding_process_time, config.CONVEYOR_CAPACITIES[0], stats)
    cleaning = ManufacturingProcess(env, "Cleaning", config.MACHINE_CAPACITIES[1],
                                    cleaning_process_time, config.CONVEYOR_CAPACITIES[1], stats)
    filling = ManufacturingProcess(env, "Filling", config.MACHINE_CAPACITIES[2],
                                   filling_process_time, config.CONVEYOR_CAPACITIES[2], stats)
    capping_labeling = ManufacturingProcess(env, "Capping & Labeling", config.MACHINE_CAPACITIES[3],
                                            capping_labeling_process_time, config.CONVEYOR_CAPACITIES[3], stats)
    packaging = ManufacturingProcess(env, "Packaging", config.MACHINE_CAPACITIES[4],
                                    packaging_process_time, config.CONVEYOR_CAPACITIES[4], stats)

    machines = [blow_molding, cleaning, filling, capping_labeling, packaging]

    def get_demand_interval():
        """Returns interarrival time for new items (demand rate)."""
        if isinstance(config.DEMAND_RATE, (int, float)):
            return config.DEMAND_RATE
        elif callable(config.DEMAND_RATE):
            return config.DEMAND_RATE()

    def production_line(env):
        """Simulates item arrivals and processing through all manufacturing stages."""
        item = 1
        # First item arrives at time 0
        print(f'Item {item} enters the system at time {env.now:.5f}')
        env.process(blow_molding.process(item))
        env.process(cleaning.process(item))
        env.process(filling.process(item))
        env.process(capping_labeling.process(item))
        env.process(packaging.process(item))
        stats['processed_items'] += 1

        next_arrival_time = get_demand_interval()

        while env.now < config.SIM_TIME:
            yield env.timeout(max(0, next_arrival_time - env.now))

            item += 1
            print(f'Item {item} enters the system at time {env.now:.5f}')
            env.process(blow_molding.process(item))
            env.process(cleaning.process(item))
            env.process(filling.process(item))
            env.process(capping_labeling.process(item))
            env.process(packaging.process(item))

            stats['processed_items'] += 1
            next_arrival_time += get_demand_interval()

    # Start and run simulation
    env.process(production_line(env))
    until = config.SIM_TIME  # Run for 90% of the simulation time
    while env.peek() < until:
        print(f"Simulation time: {env.now:.5f}")
        print(f"Processed items: {env}")
        env.step()
    # env.run(until=config.SIM_TIME)

    # Compute key performance metrics
    total_time = config.SIM_TIME
    throughput = stats['processed_items'] / (total_time / 60)  # Bottles per hour
    avg_queue_length = (sum(stats['queue_lengths']) / len(stats['queue_lengths'])) if stats['queue_lengths'] else 0
    bottleneck_station = max(stats['queue_lengths']) if stats['queue_lengths'] else 0
    machine_utilization = {
        machine.name: (machine.total_busy_time / total_time) * 100 for machine in machines
    }

    return {
        "total_bottles": stats['processed_items'],
        "throughput": throughput,
        "avg_queue_length": avg_queue_length,
        "max_queue_length": bottleneck_station,
        "utilization": machine_utilization>
    } # End of run_simulation function  


In [ ]:
all_results = []
for i in range(1):
    print(f"\n=== Running Simulation Replication {i+1} ===")
    result = run_simulation(i)
    all_results.append(result)

# 📊 Aggregate results across replications
total_bottles_list = [res["total_bottles"] for res in all_results]
throughput_list = [res["throughput"] for res in all_results]
avg_queue_list = [res["avg_queue_length"] for res in all_results]
max_queue_list = [res["max_queue_length"] for res in all_results]

# 🧮 Compute mean and standard deviation
mean_bottles, std_bottles = np.mean(total_bottles_list), np.std(total_bottles_list)
mean_throughput, std_throughput = np.mean(throughput_list), np.std(throughput_list)
mean_queue, std_queue = np.mean(avg_queue_list), np.std(avg_queue_list)
mean_max_queue, std_max_queue = np.mean(max_queue_list), np.std(max_queue_list)

# 🏭 Machine utilization (mean ± std dev)
utilization_results = {machine: [] for machine in all_results[0]["utilization"].keys()}
for res in all_results:
    for machine, util in res["utilization"].items():
        utilization_results[machine].append(util)

# 📢 Print aggregated results
print("\n=== Aggregated Simulation Results Across Replications ===")
print(f"Total Bottles Produced: Mean = {mean_bottles:.2f}, Std Dev = {std_bottles:.2f}")
print(f"Throughput: Mean = {mean_throughput:.2f} bottles/hour, Std Dev = {std_throughput:.2f}")
print(f"Average Queue Length: Mean = {mean_queue:.2f}, Std Dev = {std_queue:.2f}")
print(f"Maximum Queue Length: Mean = {mean_max_queue:.2f}, Std Dev = {std_max_queue:.2f}")

print("\nMachine Utilization (Mean ± Std Dev):")
for machine, values in utilization_results.items():
    print(f"  {machine}: {np.mean(values):.2f}% ± {np.std(values):.2f}%")

In [ ]:
# Define constants
NUM_MACHINES = 5
MACHINE_CAPACITIES = [10, 8, 12, 6, 15]  # Capacities for each machine
PROCESS_TIMES = [2, 1.5, 3, 2.5, 4]  # Processing times for each machine (in simulation time units)
INTER_ARRIVAL_TIME = 1  # Time between arrivals of unit plastics

# Machine names for clarity
MACHINE_NAMES = ["Blow Molding", "Cleaning", "Filling", "Capping/Labeling", "Packaging"]

def unit_plastic_generator(env, machines):
    """Generates unit plastics periodically and sends them to the first machine."""
    unit_id = 0
    while True:
        yield env.timeout(random.expovariate(1 / INTER_ARRIVAL_TIME))
        print(f"Time {env.now}: Unit Plastic {unit_id} created.")
        env.process(process_unit(env, f"Unit Plastic {unit_id}", machines))
        unit_id += 1

def process_unit(env, unit_name, machines):
    """Processes a unit plastic through the pipeline."""
    for i, machine in enumerate(machines):
        with machine.request() as req:
            yield req
            print(f"Time {env.now}: {unit_name} starts at {MACHINE_NAMES[i]}.")
            yield env.timeout(random.normalvariate(PROCESS_TIMES[i], PROCESS_TIMES[i] * 0.1))  # Add variability
            print(f"Time {env.now}: {unit_name} finishes at {MACHINE_NAMES[i]}.")

# Create the simulation environment
env = simpy.Environment()

# Create resources (machines) with defined capacities
machines = [simpy.Resource(env, capacity=MACHINE_CAPACITIES[i]) for i in range(NUM_MACHINES)]

# Start the unit plastic generator process
env.process(unit_plastic_generator(env, machines))

# Run the simulation for a specified duration
SIMULATION_TIME = 50
env.run(until=SIMULATION_TIME)

In [ ]:
random.seed(BASE_SEED + replication_num)

env = simpy.Environment()

# Statistics tracking for each replication
stats = {
    'processed_items': 0,  # Total completed items
    'queue_lengths': [],
    'bottleneck_station': None,
    'machine_utilization': {}
}

# Define manufacturing stages
blow_molding = ManufacturingProcess(env, "Blow Molding", config.MACHINE_CAPACITIES[0],
                                    blow_molding_process_time, config.CONVEYOR_CAPACITIES[0], stats)
cleaning = ManufacturingProcess(env, "Cleaning", config.MACHINE_CAPACITIES[1],
                                cleaning_process_time, config.CONVEYOR_CAPACITIES[1], stats)
filling = ManufacturingProcess(env, "Filling", config.MACHINE_CAPACITIES[2],
                                filling_process_time, config.CONVEYOR_CAPACITIES[2], stats)
capping_labeling = ManufacturingProcess(env, "Capping & Labeling", config.MACHINE_CAPACITIES[3],
                                        capping_labeling_process_time, config.CONVEYOR_CAPACITIES[3], stats)
packaging = ManufacturingProcess(env, "Packaging", config.MACHINE_CAPACITIES[4],
                                packaging_process_time, config.CONVEYOR_CAPACITIES[4], stats)

machines = [blow_molding, cleaning, filling, capping_labeling, packaging]

def get_demand_interval():
    """Returns interarrival time for new items (demand rate)."""
    if isinstance(config.DEMAND_RATE, (int, float)):
        return config.DEMAND_RATE
    elif callable(config.DEMAND_RATE):
        return config.DEMAND_RATE()

def production_line(env):
    """Simulates item arrivals and processing through all manufacturing stages."""
    item = 1
    # First item arrives at time 0
    print(f'Item {item} enters the system at time {env.now:.5f}')
    env.process(blow_molding.process(item))
    env.process(cleaning.process(item))
    env.process(filling.process(item))
    env.process(capping_labeling.process(item))
    env.process(packaging.process(item))
    stats['processed_items'] += 1

    next_arrival_time = get_demand_interval()

    while env.now < config.SIM_TIME:
        yield env.timeout(max(0, next_arrival_time - env.now))

        item += 1
        print(f'Item {item} enters the system at time {env.now:.5f}')
        env.process(blow_molding.process(item))
        env.process(cleaning.process(item))
        env.process(filling.process(item))
        env.process(capping_labeling.process(item))
        env.process(packaging.process(item))

        stats['processed_items'] += 1
        next_arrival_time += get_demand_interval()

# Start and run simulation
env.process(production_line(env))

## RL

In [22]:
# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    

Device set to : NVIDIA GeForce RTX 2060


In [23]:
class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, has_continuous_action_space, action_std_init):
        super(ActorCritic, self).__init__()

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_dim = action_dim
            self.action_var = torch.full((action_dim,), action_std_init * action_std_init).to(device)

        # actor
        if has_continuous_action_space :
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Tanh()
                        )
        else:
            self.actor = nn.Sequential(
                            nn.Linear(state_dim, 64),
                            nn.Tanh(),
                            nn.Linear(64, 64),
                            nn.Tanh(),
                            nn.Linear(64, action_dim),
                            nn.Softmax(dim=-1)
                        )

        
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 64),
                        nn.Tanh(),
                        nn.Linear(64, 64),
                        nn.Tanh(),
                        nn.Linear(64, 1)
                    )
        
    def set_action_std(self, new_action_std):

        if self.has_continuous_action_space:
            self.action_var = torch.full((self.action_dim,), new_action_std * new_action_std).to(device)
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling ActorCritic::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def forward(self):
        raise NotImplementedError
    

    def act(self, state):

        if self.has_continuous_action_space:
            action_mean = self.actor(state)
            cov_mat = torch.diag(self.action_var).unsqueeze(dim=0)
            dist = MultivariateNormal(action_mean, cov_mat)
        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        state_val = self.critic(state)

        return action.detach(), action_logprob.detach(), state_val.detach()
    

    def evaluate(self, state, action):

        if self.has_continuous_action_space:
            action_mean = self.actor(state)
            action_var = self.action_var.expand_as(action_mean)
            cov_mat = torch.diag_embed(action_var).to(device)
            dist = MultivariateNormal(action_mean, cov_mat)
            
            # for single action continuous environments
            if self.action_dim == 1:
                action = action.reshape(-1, self.action_dim)

        else:
            action_probs = self.actor(state)
            dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std_init=0.6):

        self.has_continuous_action_space = has_continuous_action_space

        if has_continuous_action_space:
            self.action_std = action_std_init

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, has_continuous_action_space, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def set_action_std(self, new_action_std):
        
        if self.has_continuous_action_space:
            self.action_std = new_action_std
            self.policy.set_action_std(new_action_std)
            self.policy_old.set_action_std(new_action_std)
        
        else:
            print("--------------------------------------------------------------------------------------------")
            print("WARNING : Calling PPO::set_action_std() on discrete action space policy")
            print("--------------------------------------------------------------------------------------------")


    def decay_action_std(self, action_std_decay_rate, min_action_std):
        print("--------------------------------------------------------------------------------------------")

        if self.has_continuous_action_space:
            self.action_std = self.action_std - action_std_decay_rate
            self.action_std = round(self.action_std, 4)
            if (self.action_std <= min_action_std):
                self.action_std = min_action_std
                print("setting actor output action_std to min_action_std : ", self.action_std)
            else:
                print("setting actor output action_std to : ", self.action_std)
            self.set_action_std(self.action_std)

        else:
            print("WARNING : Calling PPO::decay_action_std() on discrete action space policy")

        print("--------------------------------------------------------------------------------------------")


    def select_action(self, state):

        if self.has_continuous_action_space:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state)

            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.detach().cpu().numpy().flatten()

        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(device)
                action, action_logprob, state_val = self.policy_old.act(state)
            
            self.buffer.states.append(state)
            self.buffer.actions.append(action)
            self.buffer.logprobs.append(action_logprob)
            self.buffer.state_values.append(state_val)

            return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        old_state_values = torch.squeeze(torch.stack(self.buffer.state_values, dim=0)).detach().to(device)

        # calculate advantages
        advantages = rewards.detach() - old_state_values.detach()
        

        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5 * self.MseLoss(state_values, rewards) - 0.01 * dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        

In [ ]:
update_timestep = max_ep_len * 4      # update policy every n timesteps
K_epochs = 40               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)
# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0


# training loop
while time_step <= max_training_timesteps:
    
    state = env.reset()
    current_ep_reward = 0

    for t in range(1, max_ep_len+1):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state, reward, done, _ = env.step(action)
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # if continuous action space; then decay action std of ouput action distribution
        if has_continuous_action_space and time_step % action_std_decay_freq == 0:
            ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)

            log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
            log_f.flush()

            log_running_reward = 0
            log_running_episodes = 0

        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))

            print_running_reward = 0
            print_running_episodes = 0
            
        # save model weights
        if time_step % save_model_freq == 0:
            print("--------------------------------------------------------------------------------------------")
            print("saving model at : " + checkpoint_path)
            ppo_agent.save(checkpoint_path)
            print("model saved")
            print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
            print("--------------------------------------------------------------------------------------------")
            
        # break; if the episode is over
        if done:
            break

    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()
env.close()




# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")

In [24]:
import simpy
import random
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO

# Define constants for simulation
NUM_REPLICATIONS = 10  # Number of replications for evaluation
BASE_SEED = 42         # Base seed for reproducibility
SIM_TIME = 100         # Simulation time per episode

# RL Environment for PPO Agent
class ManufacturingRL(gym.Env):
    def __init__(self):
        super(ManufacturingRL, self).__init__()
        
        # Observation space: Queue lengths and machine utilization (normalized)
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)
        
        # Action space: Adjust machine processing times (continuous values)
        self.action_space = spaces.Box(low=0.5, high=5.0, shape=(5,), dtype=np.float32)
        
        # Initialize SimPy environment and manufacturing processes
        self.env = simpy.Environment()
        self.stats = {
            'processed_items': 0,
            'queue_lengths': [],
            'machine_utilization': {}
        }
        
        # Machine parameters: capacities and processing times
        self.machine_names = ["Blow Molding", "Cleaning", "Filling", "Capping & Labeling", "Packaging"]
        self.machines = []
        self.processing_times = [2.0] * 5  # Initial processing times
        
        for i in range(5):
            self.machines.append(
                ManufacturingProcess(self.env, self.machine_names[i], capacity=1,
                                     processing_time=self.processing_times[i],
                                     conveyor_capacity=10, stats=self.stats)
            )
        
        self.current_time = 0
    
    def reset(self):
        """Resets the environment for a new episode."""
        self.env = simpy.Environment()
        self.stats = {
            'processed_items': 0,
            'queue_lengths': [],
            'machine_utilization': {}
        }
        
        self.machines = []
        for i in range(5):
            self.machines.append(
                ManufacturingProcess(self.env, self.machine_names[i], capacity=1,
                                     processing_time=self.processing_times[i],
                                     conveyor_capacity=10, stats=self.stats)
            )
        
        self.current_time = 0
        
        return np.zeros(5)  # Initial observation (normalized values)
    
    def step(self, action):
        """Performs one step in the simulation."""
        # Update processing times based on action taken by PPO agent
        for i in range(5):
            self.machines[i].processing_time = action[i]
        
        # Run one timestep in SimPy environment
        self.env.run(until=self.current_time + 1)
        
        # Compute observations (normalized queue lengths and utilization)
        queue_lengths = [len(machine.queue.items) / 10 for machine in self.machines]
        utilization = [machine.total_busy_time / SIM_TIME for machine in self.machines]
        
        observation = np.array(queue_lengths + utilization[:5])
        
        # Compute reward based on throughput and queue lengths (example reward function)
        throughput = self.stats['processed_items'] / SIM_TIME
        reward = throughput - sum(queue_lengths) * 0.1
        
        done = self.current_time >= SIM_TIME
        
        return observation, reward, done, {}
    
    def render(self):
        """Optional render method."""
        pass

# Define ManufacturingProcess class (simplified for this example)
class ManufacturingProcess:
    def __init__(self, env, name, capacity, processing_time, conveyor_capacity, stats):
        """
        Initializes a manufacturing process with a queue and resource.
        """
        self.env = env
        self.name = name
        self.capacity = capacity  # Machine capacity (number of items it can process simultaneously)
        self.processing_time = processing_time  # Time required to process one item
        self.conveyor_capacity = conveyor_capacity  # Maximum queue length
        self.stats = stats  # Shared statistics dictionary
        
        # SimPy resources for machine and conveyor (queue)
        self.machine = simpy.Resource(env, capacity=self.capacity)
        self.queue = simpy.Store(env, capacity=self.conveyor_capacity)  # Queue with limited capacity
        
        # Tracking utilization
        self.total_busy_time = 0.0  # Total time machine is busy
        self.last_start_time = None  # For utilization calculation

    def process(self, item):
        """
        Processes an item through the machine.
        Items wait in the queue if the machine is busy.
        """
        # Add item to the queue
        yield self.queue.put(item)
        
        print(f"Time {self.env.now:.2f}: Item {item} enters {self.name} queue (Queue Length: {len(self.queue.items)}).")
        
        # Wait for machine availability
        with self.machine.request() as req:
            yield req  # Wait until machine is free
            
            # Remove item from the queue
            yield self.queue.get()
            
            print(f"Time {self.env.now:.2f}: Item {item} starts processing at {self.name}.")
            
            # Track busy time for utilization calculation
            self.last_start_time = self.env.now
            
            # Simulate processing time
            yield self.env.timeout(self.processing_time)
            
            print(f"Time {self.env.now:.2f}: Item {item} finishes processing at {self.name}.")
            
            # Update total busy time
            self.total_busy_time += self.env.now - self.last_start_time
            
            # Track queue length for statistics
            self.stats['queue_lengths'].append(len(self.queue.items))


# Train PPO Agent on ManufacturingRL Environment
def train_ppo_agent():
    env = ManufacturingRL()
    model = PPO("MlpPolicy", env, verbose=1)
    
    print("\n=== Training PPO Agent ===")
    model.learn(total_timesteps=10000)  # Train for a fixed number of timesteps
    
    return model

# Evaluate trained PPO Agent on multiple replications
def evaluate_agent(model):
    env = ManufacturingRL()
    results = []
    
    print("\n=== Evaluating PPO Agent ===")
    for _ in range(NUM_REPLICATIONS):
        obs = env.reset()
        
        done = False
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, _info = env.step(action)
        
        results.append(env.stats['processed_items'])
    
    print(f"Average Bottles Produced: {np.mean(results):.2f}")
    print(f"Standard Deviation: {np.std(results):.2f}")

In [25]:
ppo_model = train_ppo_agent()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/conangg/miniconda3/envs/agile_stem/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/conangg/miniconda3/envs/agile_stem/lib/python3.9/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(



=== Training PPO Agent ===


ValueError: could not broadcast input array from shape (10,) into shape (5,)